## Notebook 11.2: *denovo* PacBio genome assembly 


### Learning objectives: 

By the end of this notebook you will:

+ Experience running PacBio long-read assembly.
+ Compare long read assembly to short-read and hybrid assemblies from notebook 1.
+ Learn about alternative assembly software.


### Assigned readings: 

+ Li, Fay-Wei, and Alex Harkess. “A Guide to Sequence Your Favorite Plant Genomes.” Applications in Plant Sciences 6, no. 3 (n.d.): e1030. https://doi.org/10.1002/aps3.1030.


### Other resources:

**You do not need to read this entire paper, but it will be used as a reference for this notebook:**

+ Sumpter, Nicholas, Margi Butler, and Russell Poulter. “Single-Phase PacBio De Novo Assembly of the Genome of the Chytrid Fungus Batrachochytrium Dendrobatidis, a Pathogen of Amphibia.” Microbiology Resource Announcements 7, no. 21 (November 2018). https://doi.org/10.1128/MRA.01348-18.



In [ ]:
import pandas as pd

In [ ]:
# conda install bioconda::miniasm
# conda install bioconda::minimap2
# conda install bioconda::canu
# conda install bioconda::bandage

### Batrachochytrium dendrobatidis genome assembly

In this notebook we will continue to examine the published genome assembly data for *Bd* (https://www.ncbi.nlm.nih.gov/pubmed/30533847) that was recently assembled with PacBio reads. In the last notebook we assembled a related genome using short-read and hybrid short-read + PacBio data. Here we will assemble the genome using only PacBio long reads. In terms of the algorithms involved in assembly this will employ a quite different approach from the de Bruijn graph based approach used in the first notebook. We will try two different approaches, a very fast assembler called `miniasm`, and a much slower but more accurate method called `canu`. The latter is the method that the authors used in the study. Both are considered OLC assemblers (overlap-layout-consensus) which infers a Hamiltonian graph by mapping all reads against each other.

## Miniasm assembly

The miniasm assembly is performed in two steps. We first use the tool `minmap2` to calculate the overlaps among all of the pacbio reads in the fastq data file. Then we input this overlap information along with the raw data to `miniasm` which will infer a graph assembly -- the best way to uniquely connect all overlapping sequences. This involves splitting ambiguous loops in the graph and calculating the shortest path through the graph. The result can be converted to a final fasta file representing the consensus assembled contigs. 


### Overlap: get overlap of PacBio reads
Here we align all of the PacBio reads to each other. For very large datasets this can be time consuming. New algorithms (e.g., `minimap2` have *dramatically* improved the speed of this step recently. Although this step runs pretty fast it is too memory-intensive (requires many Gb of RAM) to run on the small servers that we have available on binder or codio. We will instead continue to examine pre-prepared result files. 

In [ ]:
%%bash

# minimap2 -x ava-pb -t 8 SRR7825134.fastq SRR7825134.fastq > SRR7825134.paf

This returns a result in the form of a [pairwise alignment format (.paf)](https://github.com/lh3/miniasm/blob/master/PAF.md) file. To avoid you having to compute this file I uploaded the first 20 lines of the file to a URL so you can easily look at it. Because the format is tabular we can load it as a DataFrame with pandas to look at it below. This file contains information about the positions where long reads in the data overlap each other, where each read is indicated by a unique index identifier.

In [ ]:
# URL to the snippet of the PAF file hosted online
url = ("https://raw.githubusercontent.com/genomics-course/"
       "11-assembly/master/notebooks/SRR7825134.20lines.paf")

# load the PAF file as a dataframe and return
pd.read_csv(url, sep="\t", header=None)

### Layout: assemble contigs from overlapping reads
The program `miniasm` is an OLC assembler, it takes the raw long-read data (.fastq) and the mapping information (.paf) and constructs contigs by constructing a graph that connects sequences based on the overlaps. An output is produced in [graphical fragment assembly format (.gfa)](https://github.com/GFA-spec/GFA-spec). This format can be used in some software (e.g. [Bandage](http://rrwick.github.io/Bandage)) to visualize the assembly. It contains within it the assembled contig sequence information as well as the loops and bubbles (ambiguities) in their assembly (the graph).


The code to run the `miniasm` assembler is commented below and you do not need to run it.

In [ ]:
%%bash

# miniasm -f SRR7825134.fastq SRR7825134.paf > SRR7825134.gfa

Please run the two code cells below which will call the `bandage` graphical tool  on the inferred GFA assembly graph to create a visualization of the assembly. Then run the cell below that to display the assembly graph. You can see that the assembly has resulted in several distinct and disconnected contigs, and that several of them contain bubbles. 

In [ ]:
# %%bash

# # create an assembly graph image with Bandage 
# bandage image \
#     SRR7825134.gfa \
#     SRR7825134.gfa.png \
#     --names \
#     --lengths \
#     --outline 0 \
#     --colour random \
#     --fontsize 10 \
#     --iter 4

In [ ]:
from IPython.display import Image
Image(filename='SRR7825134.gfa.png')

If we just want the best estimate of the assembly we can extract the contig sequences from this file to get a fasta file. This can be done using the unix tool `awk` to extract the lines from the `.gfa` file that start with "S" and convert them to fasta format, like below. This is a nice example to show that some bioinformatic tools still use basic unix commands, like awk, to accomplish standard text processing tasks.

In [ ]:
%%bash

# awk '$1 ~/S/ {print ">"$2"\n"$3}' SRR7825134.gfa > assembly_miniasm.fasta

### Consensus: polish the assembled contigs by mapping reads again

Technically the last step by miniasm performs both a layout and consensus step of the OLC assembly. The assembly fasta file was constructed from the overlaps among the long reads. However, it is common with long-read assemblies to apply one or more additional consensus steps that aim to polish the final base calls. Example programs for final polishing include racon and quiver. We will skip this step for now since it is often very time consuming. 

Before we analyze this genome file, let's try assembling the genome again from PacBio data using a different program.

### Canu assembly

The program [Canu](https://canu.readthedocs.io/en/latest/quick-start.html) is considered one of the best long-read assemblers, but it is much slower than the last approach we tried. While the assembly above using miniasm finished in just a few minutes, the canu assembly below took several hours. Once we get to the assembly comparisons you can decide if you think the longer run time was worth it. (Note: there are many other tools available as well, some of which can potentially be both fast and accurate).

One of the reasons canu is much slower is that it performs the full suite of steps we've discussed: pre-cleanup of reads, read overlap calculations, layout graph construction, consensus contig construction, and post-analysis contig polishing. A nice feature of canu is that you can tell it to do all of this with a single command rather than requiring multiple different programs or commands.


In [ ]:
# canu parameters explained 
# -p: prefix name for output files
# -d: output directory name
# -genomeSize: estimate of size
# -pacbio-raw: the raw fastq data

In [ ]:
%%bash
# canu -p SRR7825124 -d assembly_canu genomeSize=30m -pacbio-raw SRR7825134.fastq

While canu is running it prints a long log file to the screen explaining each step of the analysis and how much resources it is consuming, very similar to spades in the last notebook. We'll skip looking at the log file for now though and continue to comparing the resulting files. 

### Compare all four genome assemblies
Again we will use quast to assess the quality of our genome assemblies. The command below inputs the assembled contigs in fasta format of all four assemblies we've completed using four different approaches. It measures summary statistics of assembly quality and also BUSCO scores. You can read the report [here](https://genomics-course.github.io/11-assembly/notebooks/comparison/report.html).

In [ ]:
%%bash

# quast.py \
#     assembly_spades/scaffolds.fasta \
#     assembly_spades_hybrid/scaffolds.fasta \
#     assembly_miniasm.fasta \
#     assembly_canu/SRR7825124.fasta \
#     -o comparison \
#     --conserved-genes-finding \ 
#     --fungus

<div class="alert alert-success">
    <b>[1] Question:</b> 
    Which of the four assembly methods yielded the best assembly in your opinion? Which statistics did you base this off of? Why do you think the contig N50 was higher in some assemblies versus others? Why do you think the complete BUSCO score was higher in some versus others? Answer in Markdown below.   
</div>

<div class="alert alert-success">
    <b>[2] Question:</b> 
    Search on google scholar for another published genome from within the last three years. In the Markdown cell below report the citation for the publication, how large the genome is, which technology was used to sequence it, and a summary of the genome assembly (e.g., N50 and/or number of scaffolds/contigs). Try to find an example for an organism that is of interest to you.
</div>

In [ ]:
# answer here